In [372]:
import os
import pdfplumber
import pandas as pd
# os.chdir("..")
os.getcwd()

'd:\\Project\\invoice_pdf2csv'

In [373]:
amazon_bboxes = {
    
    "bill_to_name": (313, 202, 558, 243)
    
}

In [374]:
pdf_path = r"input\amazon4.pdf"
invoice = {}

In [375]:
def extract_text_from_bbox(pdf_path, bounding_box):
    """
    Extracts text from a specified bounding box within a PDF page.
    """
    extracted_text = None
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[0]  # Assuming data is on the first page

        # Use the extract_text() method with the bbox parameter
        cropped_page = page.crop(bounding_box)
        extracted_text = cropped_page.extract_text()
            
    return extracted_text

In [376]:
import camelot

stream_tables = camelot.read_pdf(pdf_path, pages='all', flavor='stream', suppress_stdout=False)
list_of_dfs = []
for i, table in enumerate(stream_tables):
    df = table.df.copy()
    list_of_dfs.append(df)

df = list_of_dfs[0]
seller_details = " ".join(df[0])
buyer_details = " ".join(df[1])


CropBox missing from /Page, defaulting to MediaBox


In [377]:
invoice["seller name"], invoice["seller_address"] = [item.strip() for item in seller_details.split("Sold By :", 1)[1].split("PAN No:", 1)[0].strip().split("*")]

In [378]:
invoice["pan_number"] = seller_details.split("PAN No:", 1)[1].split("GST Registration No:", 1)[0].strip()

In [379]:
invoice["gst_number"] = seller_details.split("GST Registration No:", 1)[1].split("Order Number:", 1)[0].strip()

In [380]:
invoice["order_id"] = seller_details.split("Order Number:", 1)[1].split("Order Date:", 1)[0].strip()

In [381]:
invoice["order_date"] = seller_details.split("Order Date:", 1)[1].strip()

In [382]:
buyer_details

'Billing Address : Ramesh Chandra Mandal  5 no. Pannajhil, Rabindra Nagar, Barasat BARASAT, WEST BENGAL, 700125  IN  State/UT Code: 19  Shipping Address :  Ramesh Chandra Mandal  Soumasnigdha Pal Flat no 12, 3rd Floor, Ramakrishna Villas, 12/6, 12/7, 18th Cross, 7th Main, NS Palya, BTM 2nd Stage BENGALURU, KARNATAKA, 560076 IN State/UT Code: 29 Place of supply: KARNATAKA Place of delivery: KARNATAKA Invoice Number : BLR8-15238 Invoice Details : KA-BLR8-676187913-2526 Invoice Date : 17.05.2025'

In [383]:
invoice["bill_to_name"] = df[1][1].strip()

In [384]:
invoice["bill_to_address"] = buyer_details.split(invoice['bill_to_name'], 1)[1].split("State/UT Code:", 1)[0].strip()

In [385]:
ship_to_name = extract_text_from_bbox(pdf_path, amazon_bboxes["bill_to_name"]).split("Shipping Address :", 1)[1].split("\n")[1:]

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


In [386]:
# ship_to_name = ship_to_name[1:]

In [387]:
invoice["ship_to_address"] = [buyer_details.split(string)[1] for string in ship_to_name][1].split("State/UT Code:", 1)[0].strip()

In [ ]:
invoice["ship_to_name"] = ship_to_name[1]

In [389]:
invoice

{'seller name': 'TIGER PUG COMMERCE PRIVATE LIMITED',
 'seller_address': 'Building 2 ( Wh 2), Plot no. 12/P2 (IT Sector),  Hitech, Defence and Aerospace Park, Devanahalli  Bengaluru, Karnataka, 562149  IN',
 'pan_number': 'AACCN8253B',
 'gst_number': '29AACCN8253B1ZI',
 'order_id': '408-6682334-6844362',
 'order_date': '17.05.2025',
 'bill_to_name': 'Ramesh Chandra Mandal',
 'bill_to_address': '5 no. Pannajhil, Rabindra Nagar, Barasat BARASAT, WEST BENGAL, 700125  IN',
 'ship_to_address': 'Flat no 12, 3rd Floor, Ramakrishna Villas, 12/6, 12/7, 18th Cross, 7th Main, NS Palya, BTM 2nd Stage BENGALURU, KARNATAKA, 560076 IN',
 'ship_to_name': 'Soumasnigdha Pal'}

In [390]:
df

,0,1
0,Sold By :,Billing Address :
1,TIGER PUG COMMERCE PRIVATE LIMITED,Ramesh Chandra Mandal
2,,
3,"* Building 2 ( Wh 2), Plot no. 12/P2 (IT Sector),","5 no. Pannajhil, Rabindra Nagar, Barasat"
4,,"BARASAT, WEST BENGAL, 700125"
5,"Hitech, Defence and Aerospace Park, Devanahalli",
6,,IN
7,"Bengaluru, Karnataka, 562149",
8,,State/UT Code: 19
9,IN,


In [391]:
invoice["invoice_number"] = buyer_details.split("Invoice Details :", 1)[1].split("Invoice Date :", 1)[0].strip()

In [392]:
invoice["invoice_date"] = buyer_details.split("Invoice Date :", 1)[1].strip()

In [393]:
all_extracted_tables = []
with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[0]
        tables = page.extract_tables()
if tables:
        for i, table_data in enumerate(tables):
                df = pd.DataFrame(table_data[1:], columns=table_data[0])
                all_extracted_tables.append(df)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


In [394]:
invoice["product_name"] = " ".join(all_extracted_tables[0]["Description"][0].split("HSN:", 1)[0].split("\n")).strip()

In [395]:
invoice["product_category"] = None

In [396]:
all_extracted_tables[0]

,Sl.\nNo,Description,Unit\nPrice,Qty,Net\nAmount,Tax\nRate,Tax\nType,Tax\nAmount,Total\nAmount
0,1,"YATWIN Silicone Back Case for iPhone 12 Mini, ...",₹254.24,1,₹254.24,9%\n9%,CGST\nSGST,₹22.88\n₹22.88,₹300.00
1,TOTAL:,None,None,None,None,None,None,₹45.76,₹300.00
2,Amount in Words:\nThree Hundred only,None,None,None,None,None,None,None,None
3,For TIGER PUG COMMERCE PRIVATE LIMITED:\nAutho...,None,None,None,None,None,None,None,None


In [397]:
invoice["product_taxable_value"] = all_extracted_tables[0]["Net\nAmount"][0].strip("₹")

In [398]:
invoice["product_taxable_value"]

'254.24'

In [399]:
invoice["GST"] = str(pd.to_numeric(all_extracted_tables[0]["Tax\nAmount"].astype(str).str.strip("₹"), errors='coerce').sum())

In [400]:
invoice["product_quantity"] = all_extracted_tables[0]["Qty"][0]

In [401]:
invoice["product_total"] = all_extracted_tables[0]["Total\nAmount"][0].strip("₹")

In [402]:
invoice

{'seller name': 'TIGER PUG COMMERCE PRIVATE LIMITED',
 'seller_address': 'Building 2 ( Wh 2), Plot no. 12/P2 (IT Sector),  Hitech, Defence and Aerospace Park, Devanahalli  Bengaluru, Karnataka, 562149  IN',
 'pan_number': 'AACCN8253B',
 'gst_number': '29AACCN8253B1ZI',
 'order_id': '408-6682334-6844362',
 'order_date': '17.05.2025',
 'bill_to_name': 'Ramesh Chandra Mandal',
 'bill_to_address': '5 no. Pannajhil, Rabindra Nagar, Barasat BARASAT, WEST BENGAL, 700125  IN',
 'ship_to_address': 'Flat no 12, 3rd Floor, Ramakrishna Villas, 12/6, 12/7, 18th Cross, 7th Main, NS Palya, BTM 2nd Stage BENGALURU, KARNATAKA, 560076 IN',
 'ship_to_name': 'Soumasnigdha Pal',
 'invoice_number': 'KA-BLR8-676187913-2526',
 'invoice_date': '17.05.2025',
 'product_name': 'YATWIN Silicone Back Case for iPhone 12 Mini, Soft-Touch Phone Back Cover for Apple iPhone 12 Mini - Lavender Grey | B0F21DZZ92 ( FGJ IP 12 Mini Grey-Indias )',
 'product_category': None,
 'product_taxable_value': '254.24',
 'GST': '45.76'